In [2]:
import dgl.data
import numpy as np
import torch
import torch.nn.functional as F
from dgl.nn import GraphConv
import dgl


Using backend: pytorch


In [3]:
dataset = dgl.data.AmazonCoBuyPhotoDataset()

g = dataset[0]
print(g.num_edges)

<bound method DGLHeteroGraph.num_edges of Graph(num_nodes=7650, num_edges=238163,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(745,), dtype=torch.float32)}
      edata_schemes={})>


In [4]:
class GCN(torch.nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats, allow_zero_in_degree=True)
        self.conv2 = GraphConv(h_feats, num_classes, allow_zero_in_degree=True)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


In [5]:
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [10]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    mask = g.in_degrees() == 0
    for e in range(1000):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[mask], labels[mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred == labels).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
#         if best_val_acc < val_acc:
#             best_val_acc = val_acc
#             best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, train acc: {:.3f}'.format(
                e, loss, train_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

/Users/rustamwarwick/miniconda3/envs/flink/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 2.079, train acc: 0.081
In epoch 5, loss: 2.041, train acc: 0.091
In epoch 10, loss: 2.005, train acc: 0.123
In epoch 15, loss: 1.971, train acc: 0.152
In epoch 20, loss: 1.939, train acc: 0.186
In epoch 25, loss: 1.910, train acc: 0.207
In epoch 30, loss: 1.883, train acc: 0.217
In epoch 35, loss: 1.859, train acc: 0.224


KeyboardInterrupt: 

[GraphConv(in=745, out=16, normalization=both, activation=None),
 GraphConv(in=16, out=8, normalization=both, activation=None)]

In [12]:
features = g.ndata['feat']
labels = g.ndata['label']

In [24]:
torch.jit.script(model)

RuntimeError: Can't redefine method: forward on class: __torch__.dgl.nn.pytorch.conv.graphconv.___torch_mangle_10.GraphConv (of Python compilation unit at: 0x7fdd69ceca58)

In [29]:
list(list(model.modules())[1].parameters())

[Parameter containing:
 tensor([[-0.0124,  0.0413, -0.0703,  ...,  0.0828, -0.0127,  0.0499],
         [ 0.0306, -0.0257, -0.0743,  ..., -0.0174,  0.0734,  0.0704],
         [ 0.0830,  0.0244, -0.0528,  ..., -0.0407, -0.0247, -0.0616],
         ...,
         [-0.0745,  0.0556,  0.0259,  ...,  0.0633,  0.0325, -0.0803],
         [-0.0004, -0.0380, -0.0207,  ...,  0.0316, -0.0254,  0.0708],
         [ 0.0353,  0.0609, -0.0208,  ..., -0.0413,  0.0412,  0.0104]],
        requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        requires_grad=True)]

In [31]:
class Test:
    @property
    def slm(self):
        return 2

In [32]:
a = Test()


AttributeError: can't set attribute

In [35]:
a = {"a":"SALAM"}


In [37]:
for key, value in a:
    print(key,value)

ValueError: not enough values to unpack (expected 2, got 1)